## Part  A - Create hash url and extract images, save downloaded images into image_folder/

### any url that didn't result in an image is listed in the not_image.txt file
### any url that resulted  in a downloaded image file, is displayed in the downloaded_files.html

In [1]:
import pandas as pd
import hashlib
import numpy as np
import requests
import re
import logging
import os
import mimetypes
import concurrent.futures
import datetime 
from PIL import Image
from io import BytesIO
from slugify import slugify
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
import cv2
from pathlib import Path
import shutil

In [2]:
# watch_file="deduplicated_A_Lange_and_Sohne_20240620.csv"
watch_file="bulk_a_lange.csv"
output_path = "image_folder/"
not_images = []
bad_url_files =[]
downloaded_images =[]   # used in populating html page
data = pd.read_csv(watch_file,header=0)
df = pd.DataFrame(data)

# Get the count of the original data rows
original_df_count=df.shape[0]
df.columns

Index(['reference_number', 'watch_url', 'type', 'brand', 'year_introduced',
       'parent_model', 'specific_model', 'nickname', 'marketing_name', 'style',
       'currency', 'price', 'image_url', 'made_in', 'case_shape',
       'case_material', 'case_finish', 'caseback', 'diameter', 'between_lugs',
       'lug_to_lug', 'case_thickness', 'bezel_material', 'bezel_color',
       'crystal', 'water_resistance', 'weight', 'dial_color', 'numerals',
       'bracelet_material', 'bracelet_color', 'clasp_type', 'movement',
       'caliber', 'power_reserve', 'frequency', 'jewels', 'features',
       'description', 'short_description'],
      dtype='object')

In [3]:
# Creating hash column will be needed later for S3 images
def get_hash(url):
        hashed_url = hashlib.md5(url.encode()).hexdigest()
        return hashed_url



In [4]:
df['hash_watch_url'] = df['watch_url'].apply(get_hash)
df['image_url'].apply(lambda x: isinstance(x, str))


0       True
1       True
2       True
3       True
4       True
        ... 
1680    True
1681    True
1682    True
1683    True
1684    True
Name: image_url, Length: 1685, dtype: bool

In [5]:
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("download_image.log"),
        logging.StreamHandler()
    ]
)



def sanitize_filename(filename):
    """
    Sanitize the filename by replacing or removing invalid characters.
    """
    invalid_chars = ['<', '>', ':', '"', '/', '\\', '|', '?', '*']
    for char in invalid_chars:
        filename = filename.replace(char, '_')  # Replace with underscore or any other valid character
    return filename

def download_image(row, index):
    """
    Download an image based on the given URL, brand, model, and SKU.
    
    Parameters:
    ----------
    image_url : url of the current image
    hash_watch_url: hashed value of current image url
    index : int
        Unique index to avoid filename collisions
    output_path : str
        Directory to save the downloaded images

    Returns:
    -------
    None
    """
    img_url = row['image_url']
    hash_url = row['hash_watch_url']
    

    # Headers to mimic a real user request
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36"
    }

    # Check if the URL is a string
    if not isinstance(img_url, str):
        bad_url_files.append(f"Skipping {hash_url} due to invalid URL.")
        return

    try:
        response = requests.get(img_url, headers=headers)
    except Exception as e:
        logging.error(f"Failed to send GET request: {e}")
        return
    
         
# Check if the request was successful
    if response.status_code == 200:  # HTTP OK
        try:
            
            # Read and save the image
            image_data = BytesIO(response.content)
            image = Image.open(image_data)                   
           
            # Extract the file extension from the URL
            file_extension = mimetypes.guess_extension(response.headers.get('content-type'))

            if file_extension:
                sanitized_filename = sanitize_filename(f"{hash_url}{file_extension}")
                
            else:
                logging.warning(f"Cannot determine file extension for {index}_{hash_url}. Saving as PNG.")
                sanitized_filename = sanitize_filename(f"{hash_url}.png")
               
            image_save_path = os.path.join(output_path, sanitized_filename)
            image.save(image_save_path)
            logging.info(f"Downloaded {sanitized_filename}")
            downloaded_images.append(f"{sanitized_filename}")
            f_image.write(f"<a href='{output_path}{sanitized_filename}'><img src='{output_path}{sanitized_filename}'>{sanitized_filename}</a> ")

        except Exception as e:
            logging.error(f"Failed to save image: {e}")
            not_images.append(f"{hash_url}")
    else:
        bad_url_files.append(f"{hash_url}")

    
    
if __name__ == "__main__":
    
    # Ensure the output directory exists, create it if not
    if os.path.exists(output_path):
        try:
            shutil.rmtree(output_path)
            print(f"Directory '{output_path}' and all its contents have been deleted.")
        except Exception as e:
            print(f"Error: Failed to delete '{output_path}'. {e}")
    os.makedirs(output_path)  
   

    with open("downloaded_files.html", "w") as f_image:     # Write downloaded images
        f_image.write("<html><head><style>img {width:100px;height:100px;background-color:#39F;}</style></head><body>")
        
        # Check if the DataFrame is not empty
        if not df.empty:
            # Use ThreadPoolExecutor to download images concurrently
            with concurrent.futures.ThreadPoolExecutor(max_workers=20) as executor:
                futures = [executor.submit(download_image, row, z) for z, row in df.iterrows()]
                concurrent.futures.wait(futures)
                
        f_image.write("</body></html>") 
        
    # Write the possible errors
    with open("not_images.txt", "w") as f:
        for item in not_images:
            f.write(f"{item}\n")
            
    with open("bad_url_files.txt", "w") as f:
        for item in bad_url_files:
            f.write(f"{item}\n")
    
    with open("downloaded_images.txt", "w") as f:
        for item in downloaded_images:
            f.write(f"{item}\n")
            
 # any image file extracted from a valid url will be stored in the images_folder created above, and documented in the downloaded_images.txt file           

Directory 'image_folder/' and all its contents have been deleted.


2024-08-01 23:23:50,160 - INFO - Downloaded c2056c5653d1c18188f33fba6dad0dc6.jpg
2024-08-01 23:23:50,192 - INFO - Downloaded b76fa1c9f7eccac7733f1cb43ca69e71.jpg
2024-08-01 23:23:50,238 - INFO - Downloaded 7bdc15af10223319c6b63d87e4ba32d2.jpg
2024-08-01 23:23:50,239 - INFO - Downloaded 68051adc159479a97d23b290a0d136e4.jpg
2024-08-01 23:23:50,245 - INFO - Downloaded cf2281af8c1a007015117ff1ff431997.jpg
2024-08-01 23:23:50,285 - INFO - Downloaded 459743f3894173708fa383dcbdad496e.jpg
2024-08-01 23:23:50,328 - INFO - Downloaded 28652ff2945a117208ca6fddecda4079.jpg
2024-08-01 23:23:50,362 - INFO - Downloaded d038845ec5d799d55e4d24d9595e0c1e.jpg
2024-08-01 23:23:50,384 - INFO - Downloaded 254489541ca7dd12e2b4c82b679795a7.jpg
2024-08-01 23:23:50,413 - INFO - Downloaded be84f6ce567aa2fab6681727fb395ba2.jpg
2024-08-01 23:23:50,430 - INFO - Downloaded 11a28b7683983a8c109fe4e495f3ed18.jpg
2024-08-01 23:23:50,441 - INFO - Downloaded 39cc6c9d3a7b6cefe17831bd27e1447c.jpg
2024-08-01 23:23:50,446 - IN

2024-08-01 23:23:52,472 - INFO - Downloaded 5ae6278ab530ceb5bebd968767ebb0ef.jpg
2024-08-01 23:23:52,493 - INFO - Downloaded 59fb484d83af8d35e4fbaa547de06227.jpg
2024-08-01 23:23:52,494 - INFO - Downloaded 7e777d428a2bbe6eb2ed1a41c0a0a285.jpg
2024-08-01 23:23:52,500 - INFO - Downloaded 431065c42ac40b705ebccf489f6202ee.jpg
2024-08-01 23:23:52,503 - INFO - Downloaded b161cfbb81cb77a9bc13afa6d9299d74.jpg
2024-08-01 23:23:52,580 - INFO - Downloaded f91c6ed4ca6f8bcc9b24b4efd4317ba0.jpg
2024-08-01 23:23:52,611 - INFO - Downloaded 6cee0411fe9f8661e4f7c86e12e0cd6c.jpg
2024-08-01 23:23:52,613 - INFO - Downloaded cbe4adbbcf7e3881b3bf5688cc178423.jpg
2024-08-01 23:23:52,621 - INFO - Downloaded 4b84c7e0fe61e7daf3df4a09a97e9a9b.jpg
2024-08-01 23:23:52,622 - INFO - Downloaded 3d4c78f1eca40e0c7cf2f3511752da5f.jpg
2024-08-01 23:23:52,625 - INFO - Downloaded b734caa48127e95f9d6dcf7ff596c06d.jpg
2024-08-01 23:23:52,628 - INFO - Downloaded 54df240275bc2ff73db46b1b43896b87.jpg
2024-08-01 23:23:52,659 - IN

2024-08-01 23:23:53,565 - INFO - Downloaded 7ebbac64362966e2b997dcb93c1feec1.jpg
2024-08-01 23:23:53,590 - INFO - Downloaded 7fc896e0e1cad584f2af5bce6c72cb11.jpg
2024-08-01 23:23:53,590 - INFO - Downloaded 959c13efd09ea3667d5ef0c5e4a4c5bf.jpg
2024-08-01 23:23:53,593 - INFO - Downloaded 0ab2c0560e9dd4754bc9f3dd7704d7b0.jpg
2024-08-01 23:23:53,619 - INFO - Downloaded ae1c83cc16f385e320a58bb4450e272d.jpg
2024-08-01 23:23:53,651 - INFO - Downloaded c252753dfa442c153e9ac48c4897e848.jpg
2024-08-01 23:23:53,656 - INFO - Downloaded 22fb2f42ca3b377a5b37921d856557a2.jpg
2024-08-01 23:23:53,672 - INFO - Downloaded 4d159a6c9df9ea1fd73628a8c2372904.jpg
2024-08-01 23:23:53,676 - INFO - Downloaded 24de136f4703e96456a4e620078586ae.jpg
2024-08-01 23:23:53,685 - INFO - Downloaded 9605512674918642099863a437269d28.jpg
2024-08-01 23:23:53,685 - INFO - Downloaded 2da6aab274400b9bc85261834fd60b96.jpg
2024-08-01 23:23:53,685 - INFO - Downloaded 234e78c9d39e41c0e676136fcd1cf32c.jpg
2024-08-01 23:23:53,688 - IN

2024-08-01 23:23:54,644 - INFO - Downloaded 4edcd3a92efe18bf513d63a6e405c5f8.jpg
2024-08-01 23:23:54,649 - INFO - Downloaded 3ac3a350509395fceeb52aa3996886a9.jpg
2024-08-01 23:23:54,660 - INFO - Downloaded 0322e101cbff9d0ad171e6a9a5aea1e7.jpg
2024-08-01 23:23:54,672 - INFO - Downloaded 864c5f75c4dbff2b05da0e6e35330b77.jpg
2024-08-01 23:23:54,687 - INFO - Downloaded d9752e57567fe5ac2d74addc8fa93584.jpg
2024-08-01 23:23:54,703 - INFO - Downloaded 2eec3b472afc5b4ce338d80b481ec57a.jpg
2024-08-01 23:23:54,705 - INFO - Downloaded 358a3c63bcbc86bbca7f42b99a8ba75b.jpg
2024-08-01 23:23:54,707 - INFO - Downloaded 4af52fb0998a71084bcc689e7e836308.jpg
2024-08-01 23:23:54,721 - INFO - Downloaded 96c80fcd97b936f7d594389c2011f385.jpg
2024-08-01 23:23:54,737 - INFO - Downloaded 3b2b3f836cde980cd344fe802c8578de.jpg
2024-08-01 23:23:54,754 - INFO - Downloaded d60dc9db0cbb5d7f93c5b24603a382f5.jpg
2024-08-01 23:23:54,756 - INFO - Downloaded fdda173b9478210ae17dd5014514f6db.jpg
2024-08-01 23:23:54,757 - IN

...

2024-08-01 23:25:54,677 - INFO - Downloaded 6805502080bf26ada7188ee04653691a.jpg
2024-08-01 23:25:54,826 - INFO - Downloaded 6c0dd1d29d6ef2910ae664c5dce16b9d.jpg
2024-08-01 23:25:54,870 - INFO - Downloaded 4f06323df63aff5cbb96bf1c5de0397b.jpg
2024-08-01 23:25:54,924 - INFO - Downloaded 102e2afe4d252ca386bb566e1797859f.jpg
2024-08-01 23:25:54,930 - INFO - Downloaded 0e9a43941d78b1da6c96ffefb6a4777d.jpg
2024-08-01 23:25:55,007 - INFO - Downloaded 234467054c67e4669c565e502dd49643.jpg
2024-08-01 23:25:55,073 - INFO - Downloaded 422c218506d9400a3e7b6923cd7ac19d.jpg
2024-08-01 23:25:55,079 - INFO - Downloaded 9864cd02bc5a95733379b19daa9ad3a5.jpg
2024-08-01 23:25:55,135 - INFO - Downloaded f7ad8ef7c90e01f2fa12883fe4d80b9f.jpg
2024-08-01 23:25:55,185 - INFO - Downloaded bffc748ed69af63bacae45df0e96ab32.jpg
2024-08-01 23:25:55,190 - INFO - Downloaded 45f95ab569fc9e16c886b1e1c25314a3.jpg
2024-08-01 23:25:55,192 - INFO - Downloaded fdd3a5a21b5d75aeb47f7e64a15fd58b.jpg
2024-08-01 23:25:55,203 - IN

2024-08-01 23:25:57,490 - INFO - Downloaded 343103ffaea33a3c5a73ac32ce646a3e.jpg
2024-08-01 23:25:57,516 - INFO - Downloaded ca9c04d839d9862a71ba59c6c677bd9a.jpg
2024-08-01 23:25:57,516 - INFO - Downloaded f64fbebfd82a9b7b70c506f8d57c6cbc.jpg
2024-08-01 23:25:57,536 - INFO - Downloaded bef8fe8540ec4483198dd1b331ea38d5.jpg
2024-08-01 23:25:57,623 - INFO - Downloaded 582d25485b25c39c599f69e7b2f706b6.jpg
2024-08-01 23:25:57,621 - INFO - Downloaded 6cf2ad602a455c519bd452c2afba48f8.jpg
2024-08-01 23:25:57,660 - INFO - Downloaded 6dd4912e3589cde45f1f6a69d59e2810.jpg
2024-08-01 23:25:57,609 - INFO - Downloaded 41f004c48530a5aaae4dee8cc5d3d239.jpg
2024-08-01 23:25:57,671 - INFO - Downloaded 9bfc696e42c1ffd26e3e69ad4fc1d3bb.jpg
2024-08-01 23:25:57,702 - INFO - Downloaded 3c2cd30ebfd536a5a466cfdf2a378c41.jpg
2024-08-01 23:25:57,783 - INFO - Downloaded a76a2b8498b8f9b398823ce9319b50d3.jpg
2024-08-01 23:25:57,787 - INFO - Downloaded 1cc1b551bd5e828afaa33aed351c5786.jpg
2024-08-01 23:25:57,823 - IN

2024-08-01 23:25:59,792 - INFO - Downloaded c6ebe36cd293868a6d8745c95596288d.jpg
2024-08-01 23:25:59,820 - INFO - Downloaded 84bb4d2d5f26629dbf25d2435f987618.jpg
2024-08-01 23:25:59,851 - INFO - Downloaded ccc4b7eaadd18ed5c0762b63dfe30355.jpg
2024-08-01 23:25:59,857 - INFO - Downloaded 67ddfad635d5d826bd32d57cc9b4364a.jpg
2024-08-01 23:25:59,869 - INFO - Downloaded d48ae56cf9b84dfd3525dc351c463706.jpg
2024-08-01 23:25:59,923 - INFO - Downloaded eee5072bfd85da82fc4dbe557503ad83.jpg
2024-08-01 23:25:59,977 - INFO - Downloaded a3d4f31e387b3ca36d17512eea508756.jpg
2024-08-01 23:25:59,978 - INFO - Downloaded 64f26435418fe17f94c78994df66021b.jpg
2024-08-01 23:25:59,982 - INFO - Downloaded 38fec1f7b9dedb815ae0115097cf7f58.jpg
2024-08-01 23:25:59,989 - INFO - Downloaded c7a8667f0d393c58c26be9c5e9ad8d54.jpg
2024-08-01 23:26:00,007 - INFO - Downloaded 7e6cedbfb7605b4dedd15dd6bf85bfdb.jpg
2024-08-01 23:26:00,024 - INFO - Downloaded 566bb5ecfcd6d5b2b38d114676f99136.jpg
2024-08-01 23:26:00,030 - IN

2024-08-01 23:26:01,914 - INFO - Downloaded 642c7547e82fd8f7e515bc23aba48f8d.jpg
2024-08-01 23:26:01,921 - INFO - Downloaded bbd683913673a095ee759449265fcafd.jpg
2024-08-01 23:26:01,993 - INFO - Downloaded 188649e02e9bcc49d503281bf8df508a.jpg
2024-08-01 23:26:02,019 - INFO - Downloaded eebc7242fa2151dd2aa882b8a53d2645.jpg
2024-08-01 23:26:02,031 - INFO - Downloaded a1633d3411d0767cd940026becf072d7.jpg
2024-08-01 23:26:02,044 - INFO - Downloaded c07bbd23769a23a759248812fbe83580.jpg
2024-08-01 23:26:02,059 - INFO - Downloaded ceb534621c2a3aa8ed0120909995a5e9.jpg
2024-08-01 23:26:02,078 - INFO - Downloaded 8911ddeb732e1caadc7a1b7a913e9a5f.jpg
2024-08-01 23:26:02,156 - INFO - Downloaded b7a74ca0e5462d9f8d5982be5262caae.jpg
2024-08-01 23:26:02,167 - INFO - Downloaded 883419622080aa57d1500539a5839e04.jpg
2024-08-01 23:26:02,173 - INFO - Downloaded 7892823a3f1d4c008054675ce797c6ad.jpg
2024-08-01 23:26:02,189 - INFO - Downloaded 3eb2cdc5cf2c66639bdbe264dba9ac66.jpg
2024-08-01 23:26:02,192 - IN

2024-08-01 23:26:04,140 - INFO - Downloaded a7d2786f13a918b3dd9036d55c99bec5.jpg
2024-08-01 23:26:04,141 - INFO - Downloaded 5e3cd04024d9c880e53c30103ca947ae.jpg
2024-08-01 23:26:04,207 - INFO - Downloaded a4fd2a6b89cdaae050eaa83ca1b3dac7.jpg
2024-08-01 23:26:04,211 - INFO - Downloaded 7af54c5d7ee1d9df5b7903da8c673cdf.jpg
2024-08-01 23:26:04,212 - INFO - Downloaded 6215cc81f6404760674a0ec0c72444be.jpg
2024-08-01 23:26:04,279 - INFO - Downloaded 989ce91cbd758c7be228b507a1f6a114.jpg
2024-08-01 23:26:04,289 - INFO - Downloaded 4f4953801d4f947d3af12c7588c912e9.jpg
2024-08-01 23:26:04,313 - INFO - Downloaded 06af7c3a9c58f0af69d7d86360f3e320.jpg
2024-08-01 23:26:04,356 - INFO - Downloaded a0d5ab5c197b3cc8e042336a20dc1d45.jpg
2024-08-01 23:26:04,319 - INFO - Downloaded 2a29f662a053a05f95cafb9b92cb6ba4.jpg
2024-08-01 23:26:04,374 - INFO - Downloaded 0db70af9d947daa9f7c20b916e99a7ff.jpg
2024-08-01 23:26:04,387 - INFO - Downloaded 8aa070f72e6b8922b6820d45f72e150a.jpg
2024-08-01 23:26:04,402 - IN

In [6]:
# Create csv file with all original records, along with new image filename as hash.png or hash.jpg

In [7]:
from datetime import date
brandname='A_Lange_and_Sohne'
today = date.today()
today_date= date.today()
formatted_date = today.strftime("%Y%m%d")
output_filename = "bulkdata_images_" + brandname +'_'+ formatted_date +'.csv'
        
df.to_csv(output_filename, index=False)
print(output_filename," file saved successfully.")

bulkdata_images_A_Lange_and_Sohne_20240801.csv  file saved successfully.


## Part B - Separate downloaded images into transparent background or otherwise

In [8]:
# Threshold Variables
BLUR_THRESHOLD = 100  # Variance of Laplacian threshold. Typical values: min=10, avg=100, max=200
EDGE_COUNT_THRESHOLD = 2000  # Edge count threshold. Typical values: min=500, avg=1000, max=2000
WHITE_BG_THRESHOLD = 240  # RGB threshold to consider a pixel as white. Typical values: min=200, avg=240, max=255
WHITE_BG_PERCENTAGE = 0.9  # Percentage of white pixels in the corners to consider background as white. Typical values: min=0.5, avg=0.9, max=1.0

def crop_to_content_with_alpha(image):
    """ 
    Crop image to content based on transparency (alpha channel).
    
    Parameters:
    image (PIL.Image): The image to be cropped.
    
    Returns:
    PIL.Image: The cropped image if transparency is present, otherwise the original image.
    """
    # Ensure the image is in RGBA mode to access the alpha channel
    if image.mode != 'RGBA':
        image = image.convert('RGBA')
    
    # Extract the alpha channel (transparency) from the image
    alpha_channel = image.split()[3]
    
    # Check if there's any transparency in the image
    if alpha_channel.getextrema()[0] < 255:
        # Get the bounding box of the non-transparent content
        bbox = alpha_channel.getbbox()
        if bbox:
            # Crop the image to the bounding box
            cropped_image = image.crop(bbox)
            return cropped_image
    
    # Return the original image if no transparency is found
    return image

def crop_to_content_without_alpha(image_path):
    """ 
    Crop image to content based on contours (for images without alpha channel).
    
    Parameters:
    image_path (str): The path to the image to be cropped.
    
    Returns:
    PIL.Image: The cropped image.
    """
    # Open the image using PIL
    image = Image.open(image_path)
    
    # Convert the PIL image to an OpenCV image (NumPy array) with alpha channel
    open_cv_image = cv2.cvtColor(np.array(image), cv2.COLOR_RGBA2BGRA)
    
    # Convert the OpenCV image to grayscale
    gray = cv2.cvtColor(open_cv_image, cv2.COLOR_BGR2GRAY)
    
    # Apply Gaussian blur to reduce noise and improve edge detection
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    
    # Perform edge detection using the Canny algorithm
    edges = cv2.Canny(blurred, 50, 150)
    
    # Dilate the edges to close gaps
    dilated = cv2.dilate(edges, None, iterations=2)
    
    # Find contours in the edge-detected image
    contours, _ = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Get the bounding box of the largest contour
    x, y, w, h = cv2.boundingRect(contours[0])
    
    # Crop the OpenCV image to the bounding box
    cropped_image = open_cv_image[y:y+h, x:x+w]
    
    # Convert the cropped OpenCV image back to RGB mode for PIL compatibility
    cropped_image_rgb = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB)
    
    # Convert the cropped image to a PIL image
    pil_cropped_image = Image.fromarray(cropped_image_rgb)
    
    # Return the cropped PIL image
    return pil_cropped_image

def is_image_vertical(image):
    """ Check if the image is vertical (portrait orientation) """
    width, height = image.size
    return height > width

def rotate_to_vertical(image):
    """ Rotate image to make it vertical if it's not already """
    if not is_image_vertical(image):
        image = image.rotate(90, expand=True)
    return image

def has_transparency(image):
    """ Check if the image has transparency (alpha channel) """
    if image.mode == 'RGBA':
        alpha_channel = image.split()[3]
        if alpha_channel.getextrema()[0] < 255:
            return True
    return False

def is_background_mostly_white(image, threshold=WHITE_BG_THRESHOLD, percentage=WHITE_BG_PERCENTAGE):
    """ Check if the background of the image is mostly white """
    image = image.convert("RGBA")
    width, height = image.size
    datas = image.getdata()

    corners = [
        (0, 0),  # Top-left
        (width - 1, 0),  # Top-right
        (0, height - 1),  # Bottom-left
        (width - 1, height - 1)  # Bottom-right
    ]

    white_pixels = 0
    total_pixels = len(corners)

    for corner in corners:
        pixel = datas[corner[1] * width + corner[0]]
        if pixel[0] > threshold and pixel[1] > threshold and pixel[2] > threshold:
            white_pixels += 1

    return (white_pixels / total_pixels) >= percentage

def is_blurry_variance_of_laplacian(image, threshold=BLUR_THRESHOLD):
    """ Check if the image is blurry using the variance of Laplacian method """
    open_cv_image = cv2.cvtColor(np.array(image), cv2.COLOR_RGBA2BGR)
    gray = cv2.cvtColor(open_cv_image, cv2.COLOR_BGR2GRAY)
    variance_of_laplacian = cv2.Laplacian(gray, cv2.CV_64F).var()
    return variance_of_laplacian < threshold

def is_blurry_edge_count(image, threshold=EDGE_COUNT_THRESHOLD):
    """ Check if the image is blurry using edge count method """
    open_cv_image = cv2.cvtColor(np.array(image), cv2.COLOR_RGBA2BGR)
    gray = cv2.cvtColor(open_cv_image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 50, 150)
    edge_count = np.count_nonzero(edges)
    return edge_count < threshold

def resize_image(cropped_image, canvas_width, canvas_height, fixed_margin):
    """ 
    Resize the image to fit within the specified canvas dimensions,
    maintaining the aspect ratio and centering the image within the canvas.
    
    Parameters:
    cropped_image (PIL.Image): The image to be resized.
    canvas_width (int): The width of the canvas.
    canvas_height (int): The height of the canvas.
    fixed_margin (int): The margin to maintain around the image.
    
    Returns:
    PIL.Image: The final image with the resized image centered on the canvas.
    """
    
    # Calculate the aspect ratio of the original image
    aspect_ratio = cropped_image.width / cropped_image.height
    
    # Calculate the maximum width and height the image can have within the canvas, accounting for margins
    max_width = canvas_width - 2 * fixed_margin
    max_height = canvas_height - 2 * fixed_margin
    
    # Determine the scale factor to resize the image while maintaining its aspect ratio
    scale_factor = min(max_width / cropped_image.width, max_height / cropped_image.height)
    
    # Calculate the new dimensions of the resized image
    new_width = int(cropped_image.width * scale_factor)
    new_height = int(cropped_image.height * scale_factor)
    
    # Resize the image using the calculated dimensions
    resized_image = cropped_image.resize((new_width, new_height), Image.LANCZOS)
    
    # Calculate the position to paste the resized image on the canvas to center it
    paste_x = (canvas_width - new_width) // 2
    paste_y = (canvas_height - new_height) // 2
    
    # Create a new transparent canvas with the specified dimensions
    final_image = Image.new("RGBA", (canvas_width, canvas_height), (0, 0, 0, 0))
    
    # Paste the resized image onto the canvas, handling transparency if present
    final_image.paste(resized_image, (paste_x, paste_y), mask=resized_image.split()[3] if "A" in resized_image.getbands() else None)
    
    # Return the final image with the resized image centered on the canvas
    return final_image

def process_image(image_path, transparent_output_folder, white_bg_output_folder, canvas_width, canvas_height, fixed_margin):
    """ Process a single image: crop, resize, and save it """
    filename = os.path.basename(image_path)
    image = Image.open(image_path).convert('RGBA')

    # Check if the image has transparency
    if has_transparency(image):
        cropped_image = crop_to_content_with_alpha(image)
        output_folder = transparent_output_folder
        print(f"Processing {filename} with transparency")
    else:
        # Check for blurriness for images without alpha
        if is_blurry_variance_of_laplacian(image):
            print(f"Skipping {filename} due to blurriness (variance of Laplacian)")
            return
        
        if is_blurry_edge_count(image):
            print(f"Skipping {filename} due to blurriness (edge count)")
            return

        # Check if the background is mostly white
        if is_background_mostly_white(image):
            cropped_image = crop_to_content_without_alpha(image_path)
            output_folder = white_bg_output_folder
            print(f"Processing {filename} with white background")
        else:
            print(f"Skipping {filename} due to non-white background")
            return
    
    
    # Rotate and resize the image
    cropped_image = rotate_to_vertical(cropped_image)
    final_image = resize_image(cropped_image, canvas_width, canvas_height, fixed_margin)
    final_image_filename =  os.path.splitext(filename)[0] + '.png' #os.path.splitext(filename)[1]
    final_image_path = os.path.join(output_folder, final_image_filename)
    final_image.save(final_image_path)

def process_images_in_folder_concurrent(input_folder, transparent_output_folder, white_bg_output_folder, canvas_width=1000, canvas_height=1000, fixed_margin=10):
    """ Process all images in a given folder concurrently """
        
       
    if os.path.exists(transparent_output_folder):
        try:
            shutil.rmtree(transparent_output_folder)
            print(f"Directory '{transparent_output_folder}' and all its contents have been deleted.")
        except Exception as e:
            print(f"Error: Failed to delete '{transparent_output_folder}'. {e}")
            
    if os.path.exists(white_bg_output_folder):
        try:
            shutil.rmtree(white_bg_output_folder)
            print(f"Directory '{white_bg_output_folder}' and all its contents have been deleted.")
        except Exception as e:
            print(f"Error: Failed to delete '{white_bg_output_folder}'. {e}")
            
 
    os.makedirs(transparent_output_folder)
    os.makedirs(white_bg_output_folder)
  

    image_files = [f for f in os.listdir(input_folder) ]
    image_paths = [os.path.join(input_folder, filename) for filename in image_files]

    with ThreadPoolExecutor(max_workers=20) as executor:
        futures = [executor.submit(process_image, image_path, transparent_output_folder, white_bg_output_folder, canvas_width, canvas_height, fixed_margin) for image_path in image_paths]
        with tqdm(total=len(futures), desc="Processing images") as progress_bar:
            for future in as_completed(futures):
                try:
                    future.result()
                except Exception as e:
                    print(f"An error occurred: {e}")
                finally:
                    progress_bar.update(1)

                    
                          

In [9]:

if __name__ == "__main__":
    input_folder = r"image_folder"
    transparent_output_folder = r"transparent_images"
    white_bg_output_folder = r"white_bg_images"
    process_images_in_folder_concurrent(input_folder, transparent_output_folder, white_bg_output_folder)


Directory 'transparent_images' and all its contents have been deleted.
Directory 'white_bg_images' and all its contents have been deleted.
Processing 011ff2d14d6b62939d9f765c1416c96f.png with transparency
Processing 00f50abe48726b58646b1e3019ba3990.png with transparency
Processing 00e7bba44cae3410c44adcfaa651405a.png with transparency
Skipping 0102a33023554dab28d7cea953ac1694.jpg due to blurriness (variance of Laplacian)
Processing 0265422981bb3f66d540ab66abf592d3.jpg with white background
Skipping 03a0f0877bab5eaa8df9a7ef132aeeef.jpg due to blurriness (variance of Laplacian)Processing 00b370de60a931180e7a68b9cca5dc4c.jpg with white background

Processing 0322e101cbff9d0ad171e6a9a5aea1e7.jpg with white background
Processing 02d816fba813ef3efc50fc522843c67b.jpg with white background
Skipping 03c1b4b1153e9000ed2f4b7567202bdd.jpg due to non-white background
Processing 015f369d469109003f4a70f5133b6799.jpg with white background
Processing 01a08adf2c8218cd60285d26b28b5be2.jpg with white back

Processing images:   0%|          | 0/1679 [00:00<?, ?it/s]

Processing 03f892aedc2abedb08008c325940502e.jpg with white background


Processing images:   1%|          | 10/1679 [00:00<01:12, 23.18it/s]

Processing 0426561ac3375863ab105cb8293d7ebc.jpg with white background
Processing 04c2989cf7b3ea3ad32b5b891449edf3.png with transparency


...

Processing images: 100%|█████████▉| 1674/1679 [02:27<00:00,  7.19it/s]

Processing fedc7824684b90c21f216e42e8ad64b5.jpg with white background


Processing images: 100%|█████████▉| 1676/1679 [02:27<00:00,  8.18it/s]

Processing ff63e6d292ad6b23be4c83673e4db47b.jpg with white background
Processing ff416ec4422659d8dbd232ca4e734647.jpg with white background


Processing images: 100%|██████████| 1679/1679 [02:28<00:00, 11.29it/s]


In [10]:
print("Total Number of original files",df.shape[0])

Total Number of original files 1685


In [11]:
folder_path = Path("white_bg_images")
num_items = len(list(folder_path.iterdir()))

print(f"Number of items in '{folder_path}': {num_items}")

Number of items in 'white_bg_images': 1116


In [12]:
folder_path = Path("transparent_images")
num_items = len(list(folder_path.iterdir()))

print(f"Number of items in '{folder_path}': {num_items}")

Number of items in 'transparent_images': 341
